In [1]:
import os
import pandas as pd
import numpy as np
import psycopg2

In [2]:
# Import csv into dataframe
url = "https://raw.githubusercontent.com/datasets/covid-19/main/data/countries-aggregated.csv"
df = pd.read_csv(url)

In [3]:
clean_tble_name = url.replace('https://raw.githubusercontent.com/datasets/covid-19/main/data/','')
print(clean_tble_name)

countries-aggregated.csv


In [4]:
df.columns

Index(['Date', 'Country', 'Confirmed', 'Recovered', 'Deaths'], dtype='object')

In [5]:
df.dtypes

Date         object
Country      object
Confirmed     int64
Recovered     int64
Deaths        int64
dtype: object

In [6]:
replacements = {
        'timedelta64[ns]': 'varchar(255)',
        'object': 'varchar(255)',
        'float64': 'float',
        'int64': 'int',
        'datetime64': 'timestamp'}

In [7]:
replaced_dtypes = df.dtypes.replace(replacements)

In [15]:
create_tble = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, replaced_dtypes))

In [17]:
print(create_tble)

Date varchar(255), Country varchar(255), Confirmed int, Recovered int, Deaths int
